# Joule API Demonstration Notebook

This notebook shows how to use the Joule Application Programming Interface (API).


Before running this notebook, you must authorize API access for the Joule user and switch to the joule kernel. 

1. Run the following command from the terminal to authorize API access:
```
  $> sudo -E joule admin authorize
  ```
  
2. Switch the kernel to ``joule`` from the menu Kernel->Change Kernel

In [ ]:
# run this cell first to import the packages
import joule
# convenience imports to make code more compact
from joule.api import EventStream, Event, DataStream, Element, Annotation
from joule.errors import EmptyPipeError
import numpy as np
from matplotlib import pyplot as plt

To use the API you must have access to the Joule node. To view accessible nodes run the following command:
    
    $> joule node list


In [ ]:
# get_node() returns the default node, add a name parameter to request a specific one
node = joule.api.get_node()

# all node methods are async so you must use the await keyword
info = await node.info()

print("Node [%s] running joule %s" % (info.name, info.version))

### Create data streams and write data

In [ ]:
# create a two element stream of 5Hz sine, cosine waveforms
freq = 5.0
t = np.arange(0,1,0.001) # 1ms sample rate
sine = np.sin(freq*2*np.pi*t)
cosine = np.cos(freq*2*np.pi*t)
tangent = np.tan(freq*2*np.pi*t)
plt.plot(t, sine, 'r', t, cosine, 'g')
plt.xlabel('Time (sec)')
plt.show()


In [ ]:
# create a stream on the Joule Node that can store this data
stream = DataStream(name="waves", elements=[Element(name="sine"), Element(name="cosine")])
stream = await node.data_stream_create(stream,"/api_demo") # now stream is a registered model and can be used with API calls

# refresh the node in the Data Explorer and you should see the new stream
# *NOTE* if you run this code more than once you will receive an error that the stream already exists

In [ ]:
# we need to put the data in an M,3 numpy array:
#  [ ts sine cosine
#    ts sine cosine
#    ...           ]
#
# There are many ways to do this, the following is rather concise
# *NOTE* make sure the timestamps are in units of microseconds
#
data = np.vstack((t*1e6, sine, cosine)).T

#
# add data to the stream by using an input pipe
#
pipe = await node.data_write(stream)
await pipe.write(data) # timestamps should be in us
await pipe.close() # make sure to close the pipe after writing



Now refresh the node in Lumen and you should see the new stream with data.

*NOTE*: If you run this code more than once you will receive an error that the data already exists since timestamps must be unique in a data stream.

### Manipulate data streams and data

In [ ]:
# get information about a stream
print("Stream Info:\t", await node.data_stream_info(stream))

# get the data intervals (regions of the stream with data)
print("Intervals:\t", await node.data_intervals(stream))

# change the display type of an element to discrete
stream.elements[1].display_type="discrete"
await node.data_stream_update(stream) # refresh the node to see this change

# remove data from a stream 
# ***DANGEROUS: OMITTING START and END will remove ALL DATA***
await node.data_delete(stream,start=0.2*1e6, end=0.4*1e6)
print("--removed data--")
print("Intervals:\t", await node.data_intervals(stream))

# ...many more methods are available, see API docs

### Data Annotations

**IMPORTANT:** Create a stream annotation in Lumen before running this cell.

In [ ]:
# retrieve a list of annotations (include start,end parameters to limit query to a time range)
annotations = await node.annotation_get(stream)

if len(annotations) == 0:
    print("ERROR: Create an annotation in Lumen then run this cell")
elif annotations[0].end is None:
        print("ERROR: Annotate a range in Lumen, not an event")
else:
    annotation = annotations[0]

    # read the data associated with the annotation
    pipe = await node.data_read(stream,start=annotation.start, end=annotation.end)
    data = await pipe.read_all() # this automatically closes the pipe

    # plot the data
    plt.plot(data['timestamp']/1e6, data['data'])
    plt.title(annotation.title)
    plt.xlabel('Time (sec)')
    plt.show()

# Annotations can also be created with the API
#

    annotation = Annotation(title='API Annotation', start=0.8*1e6)
    await node.annotation_create(annotation, stream)



Now refresh the annotations in the Plot Tab of Lumen to see this new annotation.

*NOTE*: If you run this cell multiple times it will create multiple annotations.

### Explore data streams and read data

In [ ]:
# Nodes can be explored through the API
#
root = await node.folder_root()

def print_folder(folder, indent=0):
    for child in folder.children:
        print("  "*indent + child.name)
        print_folder(child, indent+1)
    for stream in folder.data_streams:
        print("  "*indent + "[%s: %s]" % (stream.name, stream.layout))
        
# print the folder directory structure
print_folder(root)


---

*Reading Data Option 1:* `async read_all(flatten=False, maxrows=100000.0, error_on_overflow=False)→ numpy.ndarray`

https://wattsworth.net/joule/pipes.html#joule.Pipe.read_all


In [ ]:
# streams can be accessed by API object (as shown in previous cells) or by path
info = await node.data_stream_info("/api_demo/waves")
print("The demo stream has %d rows of data" % info.rows)

pipe = await node.data_read("/api_demo/waves")
data = await pipe.read_all()
print(f"retreived {len(data)} rows of data")

---

*Reading Data Option 2:*

In general you should treat a pipe as an infinite
data source and read it by chunk. This requires more code, but it scales to very large 
datasets and is the only way to work with realtime data sources

In [ ]:
# If you want to treat the data like a simple array you can use the read_all method, but if
# there is too much data this may fail. In general you should treat a pipe as an infinite
# data source and read it by chunk. This requires more code, but it scales to very large 
# datasets and is the only way to work with realtime data sources
#
print("-- reading data --")
pipe = await node.data_read("/api_demo/waves")
try:
    while True:
        data = await pipe.read()
        plt.plot(data['timestamp']/1e6,data['data'])
        pipe.consume(len(data))
        print("%d rows of data" % len(data))
        # for large data sources the chunk may or may not be an interval boundary
        # you can explicitly check whether this is the end of an interval:
        if pipe.end_of_interval:
            print(" data boundary")
except EmptyPipeError:
    pass
finally:
    await pipe.close()

plt.xlabel('Time (sec)')
plt.title('Data showing interval break')
plt.show()


## Event Streams

Event streams store unstructured JSON data with start and end timestamps

In [ ]:
# create a stream on the Joule Node that can store this data
stream = EventStream(name="Events")
stream = await node.event_stream_create(stream,"/api_demo") # now stream is a registered model and can be used with API calls

# refresh the node in the Data Explorer and you should see the new stream
# *NOTE* if you run this code more than once you will receive an error that the stream already exists

In [ ]:
# events have a start, end, and content which can contain any JSON serializable data
event1 = Event(start_time=0, end_time=0.1e6, 
               content={'height':1,'color':'red','description':'first one'})
event2 = Event(start_time=.3e6, end_time=0.5e6, 
               content={'height':2,'color':'blue','description':'second one'})

await node.event_stream_write(stream, [event1, event2])


### Reset the Node to original state
**Run this cell to undo all changes created by this notebook**

In [ ]:
await node.folder_delete("/api_demo")